In [1]:
pip install opencv-python face-recognition


Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2
import face_recognition
import numpy as np
import pickle
import os
import csv
from datetime import datetime

# Define file paths
ENCODINGS_FILE = 'face_encodings.pkl'
ATTENDANCE_FILE = 'attendance.csv'

# Initialize variables
known_face_encodings = []
known_face_names = []

# Step 1: Load encodings from file if it exists
def load_known_faces():
    if os.path.exists(ENCODINGS_FILE):
        with open(ENCODINGS_FILE, 'rb') as f:
            data = pickle.load(f)
            return data.get('encodings', []), data.get('names', [])
    return [], []

# Step 2: Save encodings to file
def save_known_faces(encodings, names):
    data = {'encodings': encodings, 'names': names}
    with open(ENCODINGS_FILE, 'wb') as f:
        pickle.dump(data, f)

# Step 3: Register a face
def register_face():
    global known_face_encodings, known_face_names
    known_face_encodings, known_face_names = load_known_faces()

    video_capture = cv2.VideoCapture(0)

    if not video_capture.isOpened():
        print("Error: Could not open video device.")
        return

    print("Registering your face. Press 'Q' to capture.")
    while True:
        ret, frame = video_capture.read()
        if not ret:
            print("Error: Failed to capture image. Retrying...")
            continue

        cv2.imshow('Register your face, press Q to capture', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    face_locations = face_recognition.face_locations(rgb_frame)
    if face_locations:
        face_encoding = face_recognition.face_encodings(rgb_frame, face_locations)[0]
        name = input("Enter your name: ").strip()

        if name == "":
            print("Name cannot be empty. Registration canceled.")
        else:
            known_face_encodings.append(face_encoding)
            known_face_names.append(name)
            print(f"Face registered for {name}.")
            save_known_faces(known_face_encodings, known_face_names)
    else:
        print("No face detected. Please try again.")

    video_capture.release()
    cv2.destroyAllWindows()

# Helper function to mark attendance
def mark_attendance(name):
    now = datetime.now()
    timestamp = now.strftime("%Y-%m-%d %H:%M:%S")
    # Check if attendance file exists; if not, create it with headers
    file_exists = os.path.isfile(ATTENDANCE_FILE)
    with open(ATTENDANCE_FILE, 'a', newline='') as csvfile:
        fieldnames = ['Name', 'Timestamp']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        if not file_exists:
            writer.writeheader()
        writer.writerow({'Name': name, 'Timestamp': timestamp})
    print(f"Attendance marked for {name} at {timestamp}.")

# Step 4: Real-time face recognition and attendance
def recognize_face():
    global known_face_encodings, known_face_names
    known_face_encodings, known_face_names = load_known_faces()

    if not known_face_names:
        print("No registered faces found. Please register faces first.")
        return

    video_capture = cv2.VideoCapture(0)

    if not video_capture.isOpened():
        print("Error: Could not open video device.")
        return

    print("Starting real-time face recognition. Press 'Q' to quit.")
    already_marked = set()  # To prevent multiple entries for the same session

    while True:
        ret, frame = video_capture.read()
        if not ret:
            print("Error: Failed to capture frame. Retrying...")
            continue

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb_frame)
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            # Draw bounding box and label
            color = (0, 255, 0) if name != "Unknown" else (0, 0, 255)
            cv2.rectangle(frame, (left, top), (right, bottom), color, 2)
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), color, cv2.FILLED)
            cv2.putText(frame, name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_DUPLEX, 1.0, (255, 255, 255), 1)

            # Mark attendance if recognized and not already marked in this session
            if name != "Unknown" and name not in already_marked:
                mark_attendance(name)
                already_marked.add(name)

        cv2.imshow('Real-time Face Recognition', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    video_capture.release()
    cv2.destroyAllWindows()

# Step 5: Show attendance
def show_attendance():
    if not os.path.exists(ATTENDANCE_FILE):
        print("No attendance records found.")
        return

    print("\nAttendance Records:")
    with open(ATTENDANCE_FILE, 'r', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            print(f"Name: {row['Name']}, Time: {row['Timestamp']}")
    print("")  # Add an empty line for better readability

# Main menu
def main_menu():
    while True:
        print("\n--- Attendance System Menu ---")
        print("1. Register a new face")
        print("2. Start face recognition (Mark Attendance)")
        print("3. Show Attendance")
        print("4. Exit")
        choice = input("Enter your choice (1-4): ").strip()

        if choice == '1':
            register_face()
        elif choice == '2':
            recognize_face()
        elif choice == '3':
            show_attendance()
        elif choice == '4':
            print("Exiting the system. Goodbye!")
            break
        else:
            print("Invalid choice. Please select a valid option.")

# Entry point
if __name__ == "__main__":
    main_menu()



--- Attendance System Menu ---
1. Register a new face
2. Start face recognition (Mark Attendance)
3. Show Attendance
4. Exit
Enter your choice (1-4): 1
Registering your face. Press 'Q' to capture.
Enter your name: aadhya
Face registered for aadhya.

--- Attendance System Menu ---
1. Register a new face
2. Start face recognition (Mark Attendance)
3. Show Attendance
4. Exit
Enter your choice (1-4): 2
Starting real-time face recognition. Press 'Q' to quit.
Attendance marked for aadhya at 2024-09-22 22:40:23.
Attendance marked for Aadhya at 2024-09-22 22:40:26.

--- Attendance System Menu ---
1. Register a new face
2. Start face recognition (Mark Attendance)
3. Show Attendance
4. Exit
Enter your choice (1-4): 3

Attendance Records:
Name: Shrihari, Time: 2024-09-22 15:38:00
Name: Aadhya, Time: 2024-09-22 15:39:20
Name: aadhya, Time: 2024-09-22 22:40:23
Name: Aadhya, Time: 2024-09-22 22:40:26


--- Attendance System Menu ---
1. Register a new face
2. Start face recognition (Mark Attendance)